In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread, imsave

from skimage.feature import register_translation
from skimage.feature.register_translation import _upsampled_dft
from scipy.ndimage import fourier_shift, shift

In [131]:
#image = np.load('normed.npy')
image = np.load('normed_fixed.npy')
#test = imread('normed.tif')

In [135]:
#plt.imshow(image[780:840, 615:675], vmax = 700)
plt.imshow(image[780+84:840+84, 615+67:675+67], vmax = 700)
plt.show()

In [137]:
# Throw away everything that isn't region around beamstop
#center = np.zeros(image.shape)
#center[780:840, 615:675] = 1 # Mask around center region with beam stop
#center = np.multiply(center, image) # Take only center region with beamstop
#center = np.nan_to_num(center) #Remove nans
#center = image[780:840, 615:675]
center = image[780+84:840+84, 615+67:675+67]

In [138]:
plt.imshow(center)
plt.show()

In [139]:
from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte

In [140]:
hough_radii = np.arange(20, 35, 2)
hough_res = hough_circle(center, hough_radii)

In [141]:
# Select the most prominent 5 circles
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=3)

In [142]:
# Draw them
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4))
cent = color.gray2rgb(center)
for center_y, center_x, radius in zip(cy, cx, radii):
    circy, circx = circle_perimeter(center_y, center_x, radius)
    cent[circy, circx] = (220, 20, 20)
ax.imshow(cent, cmap=plt.cm.gray)
plt.show()

In [439]:
# Center of image
x_pos, y_pos = np.mean(cx), np.mean(cy)
x_pos, y_pos = int(x_pos+780+84), int(y_pos+615+67)

In [440]:
x_pos, y_pos

(894, 712)

In [145]:
test = np.copy(image)
test[x_pos, y_pos] = 1000000

In [162]:
plt.imshow(test, vmax = 700)
plt.show()

In [163]:
img = np.zeros(test.shape)
rr, cc = circle_perimeter(x_pos, y_pos, 20)
img[rr, cc] = 1000
plt.imshow(np.add(image, img), vmax = 70)
plt.show()

In [207]:
# Center position of detector
imcent = np.ceil(np.divide((image.shape), 2))
# Difference between center of image and beam stop position
pos_diff = np.int(np.abs(x_pos-imcent[0])), np.int(np.abs(y_pos-imcent[1]))
# Pad image with difference
padded = np.pad(image, ((pos_diff[0], pos_diff[0]), (pos_diff[1], pos_diff[1])) , 'constant', constant_values = ((np.nan,np.nan)))
# Shift image over to move beamstop to center
shifted = shift(padded, (pos_diff[0], pos_diff[1]), order = 1, mode='constant', cval=np.nan)

In [263]:
np.ceil(np.divide((shifted.shape), 2))

array([954., 898.])

In [357]:
#shifted[954,898] = 10000
plt.imshow(shifted, vmax = 700)
plt.show()

In [265]:
np.save('centered.npy', shifted)

In [269]:
from skimage.measure import block_reduce

In [435]:
downsampled = block_reduce(shifted, block_size = (4, 4), func = np.nanmedian)
downsampled.shape

C:\Users\User\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3250: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


(477, 449)

In [437]:
plt.imshow(downsampled, vmax = 800)
plt.show()

In [438]:
np.save('downsampled.npy', downsampled)

In [422]:
def crop_center(img,cropx,cropy):
    x, y = img.shape
    centx = np.int(np.ceil(x/2)) # Center in x direction
    centy = np.int(np.ceil(y/2)) # Center in y direction
    cropx = np.int(np.floor(cropx/2)) # Crop distance in x direction
    cropy = np.int(np.floor(cropy/2)) # Crop distance in y direction
    #print(str(centx-cropx)+':'+str(centx+cropx+1), str(centy-cropy)+':'+str(centy+cropy+1))
    return img[centx-cropx:centx+cropx+1, centy-cropy:centy+cropy+1] # Crop such that center is still in correct place

In [434]:
cropped = crop_center(downsampled, 445, 425)
cropped[int(np.floor(445/2)), int(np.floor(425/2))] = 1000
plt.imshow(cropped, vmax = 700)
plt.show()

In [432]:
int(np.floor(445/2)), int(np.floor(425/2))

(223, 213)